In [1]:
import cv2
import imutils
import matplotlib.pyplot as plt
import numpy as np
import os
import torch

from imutils.video import FPS
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
from Database_Utils import WorkersDatabase

In [2]:
label_to_name = {
    0: "Emre",  # 0 etiketinin ismi
    1: "Kubilay"    # 1 etiketinin ismi
}

In [3]:
db = WorkersDatabase(db_name="Workers")

In [4]:
class StaySafe():
    def __init__(self, Model_Name: str, face_model_path: str, db_name, width = 1280, height = 1280):
        self.Model_Name = Model_Name
        self.face_model_name = face_model_path
        #self.Source = Source
        self.width = width
        self.height = height
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 🔹 GPU Kullanımı
        self.model = self.CreateYoloModel()
        self.face_model = self.CreateFaceRecognitionModel()
        self.face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.predicted_names  = []
        #self.detector = dlib.get_frontal_face_detector()
        self.database = db_name
        

    def CreateFaceRecognitionModel(self):
        face_model = torch.load(self.face_model_name, weights_only=False, map_location=self.device)
        face_model.to(self.device)
        face_model.eval()
        return face_model
    
    def CreateYoloModel(self): 
        # Çalışıyor
        """
        Aynı dizinde olan modeli parametre olarak verebiliriz. (Geliştirilecek)
        """
        model = YOLO(self.Model_Name)
        return model
    
    def recognize(self, img, labels=label_to_name):
        self.predicted_names = []  # 🔹 Önceki isimleri temizle
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        faces = self.face_detector.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(50, 50))

        for face in faces:
            x, y, w, h = face
            face_roi = img[y:y+h, x:x+w]

            resized = cv2.resize(face_roi, (128, 128))
            normalize = resized / 255.0
            tensor_image = torch.tensor(normalize, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(self.device)

            with torch.no_grad():
                result = self.face_model(tensor_image)

            _, label = torch.max(result, 1)
            predicted_label = label[0].item()
            predicted_name = labels.get(predicted_label, "Bilinmeyen")
            self.predicted_names.append(predicted_name)  # 🔹 Listeye ekleme

        return img, self.predicted_names  # 🔹 Liste olarak dön
    
    # def findWorker(self):
    #     if not self.predicted_names:
    #         return ["Boyle bir calisan bulunamadi."]  # 🔹 Boş liste yerine anlamlı dönüş
    #     workers = []
    #     for name in self.predicted_names:
    #         try:
    #             worker = db.find_employee(name=name)
    #         except:
    #             worker = "Boyle bir calisan bulunamadi."
    #         workers.append(worker)
    #     return workers
        
    def ImageDetection(self, image_path):
        """
        Verilen görsel dosya yolunu model ile tahminler ve görselleştirir.
        Args:
            image_path (str): Görsel dosya yolu.
            save_path (str, optional): Kaydedilecek dosya yolu. Varsayilan None.
        """
        # Görüntüyü yükle
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError("Görüntü yüklenemedi. Lütfen dosya yolunu kontrol edin.")

        # Model ile tahmin yap
        results = self.model(image)

        # Sınıf isimlerini al
        class_names = self.model.names

        # Tespit edilen tüm nesneleri al
        boxes = results[0].boxes

        # Her bir nesne için bounding box ve etiket çiz
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box koordinatları
            conf = box.conf[0].item()  # Güven skoru
            cls = int(box.cls[0])  # Sınıf ID'si
            label = class_names[cls]  # Sınıf ismi
            text = f"{label} {conf:.2f}"

            # Bounding box ve etiketi çiz
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Yeşil kutu
            cv2.putText(image, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)  # Etiket

        # Görüntüyü matplotlib ile göster
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # OpenCV BGR -> RGB
        plt.imshow(image_rgb)
        plt.axis('off')  # Eksenleri kaldır
        plt.show()
    
    def LiveDetection(self, Source):

        # Çalışıyor

        """
        Tracker'ın çalışıp çalışmadığını kontrol etmek için kullanılır.
        Modelin tespit edeceği nesneleri gösterir.
        """

        """
        args:
        Source: Video dosyasının yolu
        Video path yerine 0 yazılırsa webcam'den alır.
        """
        cap = cv2.VideoCapture(Source)
        while cap.isOpened():
            success, frame = cap.read(Source)
            frame = cv2.flip(frame, 1)
            frame = imutils.resize(frame, width=self.width) # pip install imutils
            if not success:
                break

            # Model ile nesne takibi yap
            results = self.model.track(frame, persist=True, verbose=False)  # persist=True: ID'leri korur ancak çizimde kullanmayacağız, verbose = Bilgi
            
            # Detayları çizmek için bounding box bilgilerini al
            for r in results:
                for box in r.boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box koordinatları
                    conf = box.conf[0].item()  # Güven skoru
                    conf_text = f"%{conf * 100:.2f}"
                    cls = int(box.cls[0])  # Sınıf ID'si
                    label = self.model.names[cls]  # Sınıf ismi
                    text = f"{label} {conf_text}"

                    # Güven skoru belirli bir seviyenin üstündeyse çiz
                    if conf > 0.5:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Yeşil kutu çiz
                        cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Sınıf ismi

            # Sonucu göster
            cv2.imshow("YOLOv8 Tracking (No ID)", frame)

            # 'q' tuşuna basarak çıkış yap
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

    def SafetyCheckFromImage(self, image_path):
        # Görüntüyü PIL ile yükle
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # Model ile tahmin yap
        results = self.model(image)

        # Sınıf isimlerini al
        class_names = self.model.names

        # Tespit edilen tüm nesneleri al
        boxes = results[0].boxes

        # Tüm person'ları bul
        persons = [box for box in boxes if class_names[int(box.cls)] == 'person']

        # Her bir person için helmet ve vest kontrolü yap
        for person in persons:
            x1, y1, x2, y2 = map(int, person.xyxy[0])  # Person'un bounding box koordinatlarını al
            has_helmet = False
            has_vest = False

            # Diğer tüm nesneleri kontrol et
            for other_box in boxes:
                other_class_id = int(other_box.cls)
                other_class_name = class_names[other_class_id]
                other_x1, other_y1, other_x2, other_y2 = map(int, other_box.xyxy[0])

                # Eğer helmet veya vest bu person ile aynı bölgede ise
                if (other_class_name == 'helmet' or other_class_name == 'vest') and \
                (other_x1 > x1 and other_x2 < x2 and other_y1 > y1 and other_y2 < y2):
                    if other_class_name == 'helmet':
                        has_helmet = True
                    elif other_class_name == 'vest':
                        has_vest = True

            # Eğer hem helmet hem de vest varsa, person'u "Safe" olarak işaretle
            if has_helmet and has_vest:
                draw.rectangle([x1, y1, x2, y2], outline="green", width=2)  # Yeşil bbox
                draw.text((x1, y1 - 20), "Safe", fill="green")  # "Safe" yazısı
            # Eğer helmet veya vest yoksa, person'u "Unsafe" olarak işaretle
            else:
                draw.rectangle([x1, y1, x2, y2], outline="red", width=2)  # Kırmızı bbox
                draw.text((x1, y1 - 20), "Unsafe", fill="red")  # "Unsafe" yazısı

        # Görüntüyü matplotlib ile göster
        plt.imshow(image)
        plt.axis('off')  # Eksenleri kaldır
        plt.show()
        
    def SafetyDetector(self, Source, recognition=False):

        
        """Webcam veya video dosyası açarak nesne tespiti yapar ve sonucu gösterir."""
        """
        args:
        Source: Video path || 0 (Kamera)
        """
        cap = cv2.VideoCapture(Source)
        
        while cap.isOpened():
            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)  # Görüntüyü yatay olarak çevir
            frame = imutils.resize(frame, self.width)
            if not ret:
                break

            # Model ile tahmin yap
            results = self.model(frame)
            class_names = self.model.names
            boxes = results[0].boxes

            # Tüm person'ları bul
            persons = [box for box in boxes if class_names[int(box.cls)] == 'person']

            # Her bir person için helmet ve vest kontrolü yap
            for person in persons:
                x1, y1, x2, y2 = map(int, person.xyxy[0])
                has_helmet = False
                has_vest = False

                for other_box in boxes:
                    other_class_id = int(other_box.cls)
                    other_class_name = class_names[other_class_id]
                    other_x1, other_y1, other_x2, other_y2 = map(int, other_box.xyxy[0])

                    if (other_class_name == 'helmet' or other_class_name == 'vest') and \
                    (other_x1 > x1 and other_x2 < x2 and other_y1 > y1 and other_y2 < y2):
                        if other_class_name == 'helmet':
                            has_helmet = True
                        elif other_class_name == 'vest':
                            has_vest = True

                # Eğer hem helmet hem de vest varsa, "Safe" olarak işaretle
                if has_helmet or has_vest:
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Yeşil bbox
                    cv2.putText(frame, 'Safe', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                # Eğer helmet veya vest yoksa, "Unsafe" olarak işaretle
                else:
                    if recognition:
                        _, workers = self.recognize(frame, label_to_name)
                        worker = [w for w in workers]
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Kırmızı bbox
                        cv2.putText(frame, f'{workers}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
                    else:
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Kırmızı bbox
                        cv2.putText(frame, 'Unsafe', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)

            # Görüntüyü yeniden boyutlandır (opsiyonel)
            resized_frame = cv2.resize(frame, (800, 600))  # 800x600 piksel boyutuna getir   

            # Sonucu göster
            cv2.imshow('Result', resized_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):  # Çıkmak için 'q' tuşuna basın
                break

        cap.release()
        cv2.destroyAllWindows()

In [5]:
stay_safe = StaySafe(Model_Name="C:/Users/celik/Desktop/ss2/Models/Yolo11n_50_epoch.pt", face_model_path="C:/Users/celik/Desktop\/ss2/Models/emre_kubilay_100_epoch_resnet.pth", db_name="Workers.db")

In [ ]:
safetyCheck_live = stay_safe.SafetyDetector(Source=0, recognition=True)